In [1]:
import numpy as np
import nibabel as nib

import os
import shutil
from glob import glob

# Utils

In [15]:
def exists(obj):
    return obj is not None

def glob_dirs(regex: str):
    """Returns a list of directories that follow `regex`."""
    return [p for p in glob(regex) if os.path.isdir(p)]

def get_first_match_for_substring(list_, substring: str):
    """Returns the first element in `list_` that matches."""
    match = None
    for elem in list_:
        if substring in elem:
            match = elem
            break
    return match

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

def get_zeros_mask_for_normal_cases(nifti):
    data = nifti.get_fdata()
    if data.ndim == 3:
        zeros_mask = np.zeros_like(data).astype(np.float32)
    elif data.ndim > 3:
        zeros_mask = np.zeros(data.shape[:3]).astype(np.float32)
    else:
        raise ValueError(f"Data has {data.ndim} dimensions, expected 3 or 4.")

    # print("Zeros mask shape:", zeros_mask.shape)

    return nib.Nifti1Image(
        zeros_mask, 
        nifti.affine, 
        nifti.header
    )

# Move the patients from ISBI_FOSCAL to MICCAI

In [7]:
src_dir = "/data/Datasets/stroke/ISBI_FOSCAL/"
dest_dir = "/data/Datasets/stroke/MICCAI/"
train_dest_dir = os.path.join(dest_dir, "train")
test_dest_dir = os.path.join(dest_dir, "test")

# Txt files with the patients to be moved to the train and test folders.
train_txt = "/home/sangohe/projects/isbi2023-foscal/splits/train_patients.txt"
test_txt = "/home/sangohe/projects/isbi2023-foscal/splits/test_patients.txt"

with open(train_txt, "r") as f:
    train_patients = f.read().splitlines()
    
    for train_patient in train_patients:
        train_patient_dir = os.path.join(src_dir, train_patient)
        train_patient_dest_dir = os.path.join(train_dest_dir, train_patient)
        os.makedirs(train_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(train_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(masks_dir))
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)

        shutil.copy(os.path.join(adc_dir, "ADC.nii.gz"), adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        copytree(masks_dir, masks_dest_dir, ignore=shutil.ignore_patterns('*.nii', "*NCCT*"))


with open(test_txt, "r") as f:
    test_patients = f.read().splitlines()
    
    for test_patient in test_patients:
        test_patient_dir = os.path.join(src_dir, test_patient)
        test_patient_dest_dir = os.path.join(test_dest_dir, test_patient)
        os.makedirs(test_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(test_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(masks_dir))
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)

        shutil.copy(os.path.join(adc_dir, "ADC.nii.gz"), adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        copytree(masks_dir, masks_dest_dir, ignore=shutil.ignore_patterns('*.nii', "*NCCT*"))

# Move normal patients

In [3]:
normal_train_patients = ["ACV-107", "ACV-110", "ACV-128", "ACV-129", "ACV-140"]
normal_test_patients = ["ACV-149", "ACV-198"]

In [24]:
src_dir = "/data/Datasets/stroke/ISBI_FOSCAL/"
dest_dir = "/data/Datasets/stroke/MICCAI/"
train_dest_dir = os.path.join(dest_dir, "train")
test_dest_dir = os.path.join(dest_dir, "test")

# Txt files with the patients to be moved to the train and test folders.
train_txt = "/home/sangohe/projects/isbi2023-foscal/splits/train_patients_2.txt"
test_txt = "/home/sangohe/projects/isbi2023-foscal/splits/test_patients_2.txt"

with open(train_txt, "r") as f:
    train_patients = f.read().splitlines()
    for train_patient in train_patients:
        train_patient_dir = os.path.join(src_dir, train_patient)
        train_patient_dest_dir = os.path.join(train_dest_dir, train_patient)
        os.makedirs(train_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(train_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(masks_dir))
        daniel_masks_dest_dir = os.path.join(masks_dest_dir, "Daniel")
        andres_masks_dest_dir = os.path.join(masks_dest_dir, "Andres")
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)
        os.makedirs(daniel_masks_dest_dir, exist_ok=True)
        os.makedirs(andres_masks_dest_dir, exist_ok=True)

        adc_match = os.path.join(adc_dir, "ADC.nii.gz")
        shutil.copy(adc_match, adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        
        # Create and save zero masks for the normal cases for both radiologists.
        adc = nib.load(adc_match)
        dwi = nib.load(dwi_match)
        adc_mask = get_zeros_mask_for_normal_cases(adc)
        dwi_mask = get_zeros_mask_for_normal_cases(dwi)
        nib.save(adc_mask, os.path.join(daniel_masks_dest_dir, "ADC.nii.gz"))
        nib.save(dwi_mask, os.path.join(daniel_masks_dest_dir, "DWI.nii.gz"))
        nib.save(adc_mask, os.path.join(andres_masks_dest_dir, "ADC.nii.gz"))
        nib.save(dwi_mask, os.path.join(andres_masks_dest_dir, "DWI.nii.gz"))

with open(test_txt, "r") as f:
    test_patients = f.read().splitlines()
    for test_patient in test_patients:
        test_patient_dir = os.path.join(src_dir, test_patient)
        test_patient_dest_dir = os.path.join(test_dest_dir, test_patient)
        os.makedirs(test_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(test_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(masks_dir))
        daniel_masks_dest_dir = os.path.join(masks_dest_dir, "Daniel")
        andres_masks_dest_dir = os.path.join(masks_dest_dir, "Andres")
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)
        os.makedirs(daniel_masks_dest_dir, exist_ok=True)
        os.makedirs(andres_masks_dest_dir, exist_ok=True)

        adc_match = os.path.join(adc_dir, "ADC.nii.gz")
        shutil.copy(adc_match, adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        
        # # Create and save zero masks for the normal cases for both radiologists.
        adc = nib.load(adc_match)
        dwi = nib.load(dwi_match)
        adc_mask = get_zeros_mask_for_normal_cases(adc)
        dwi_mask = get_zeros_mask_for_normal_cases(dwi)
        nib.save(adc_mask, os.path.join(daniel_masks_dest_dir, "ADC.nii.gz"))
        nib.save(dwi_mask, os.path.join(daniel_masks_dest_dir, "DWI.nii.gz"))
        nib.save(adc_mask, os.path.join(andres_masks_dest_dir, "ADC.nii.gz"))
        nib.save(dwi_mask, os.path.join(andres_masks_dest_dir, "DWI.nii.gz"))

Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 26)
Zeros mask shape: (256, 256, 24)
Zeros mask shape: (256, 256, 24)


In [25]:
(nib.load("/data/Datasets/stroke/MICCAI/train/ACV-107/Masks/Daniel/ADC.nii.gz").shape,
nib.load("/data/Datasets/stroke/MICCAI/train/ACV-107/Masks/Andres/ADC.nii.gz").shape,
nib.load("/data/Datasets/stroke/MICCAI/train/ACV-107/Masks/Daniel/DWI.nii.gz").shape,
nib.load("/data/Datasets/stroke/MICCAI/train/ACV-107/Masks/Andres/DWI.nii.gz").shape)

((256, 256, 24), (256, 256, 24), (256, 256, 24), (256, 256, 24))